## Nimeüksused lausetes test5

In [117]:
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from collections import Counter
import itertools
from copy import deepcopy

Tühi järjend lausete jaoks:

In [45]:
corpus = []

Tühi järjend lausete ID-de jaoks:

In [46]:
sentence_ids = []

Andmebaasist loetakse sisse laused:

In [47]:
con = sqlite3.connect("media_data.db")
cur = con.cursor()

In [49]:
for row in cur.execute("SELECT ID, sentence FROM sentences WHERE named_entity = 1"):
    sentence_id = row[0]
    sentence = json_to_text(json_text=row[1])
    corpus.append(sentence)
    sentence_ids.append(sentence_id)

In [50]:
con.close()

Vaatame üht lauset ja selle nimeüksusi:

In [51]:
for ner in corpus[5].ner:
    print(ner.nertag, ner.text, ner.enclosing_text)

LOC ['Eestis'] Eestis
PER ['Eerik'] Eerik


NerTagger võib olla teinud vigu, näiteks märkinud isikunime asukohaks, asukoha organisatsiooniks jne. Siin katsetan lihtsakoelist lahendust, mis ei korrigeeri kindlasti kõiki selliseid vigu, kuid võib nende vigade hulka vähendada. Nimelt loetakse kokku eri nimeüksuste esinemised kõigis liikides. Kui nimeüksus esineb rohkem kui ühes liigis, loetakse õigeks liigiks see, milles tema esinemissagedus kõige suurem on. Kui esinemissagedused on eri liikides võrdsed, ei tehta aga midagi.

Esiteks loon tühjad järjendid nii isikunimede, organisatsiooninimede kui asukohanimede jaoks:

In [72]:
entities_PER = []
entities_ORG = []
entities_LOC = []

Seejärel käin läbi kõik korpuses olevad laused ja lisan nimeüksused järjenditesse vastavalt nende liigile:

In [73]:
for sentence in corpus:
    for ner in sentence.ner:
        lemmatized_ner = [word.lemma[0] for word in ner]
        if ner.nertag == 'PER':
            entities_PER.append(' '.join(lemmatized_ner))
        elif ner.nertag == 'ORG':
            entities_ORG.append(' '.join(lemmatized_ner))
        else:
            entities_LOC.append(' '.join(lemmatized_ner))

In [99]:
len(entities_PER)

320

In [100]:
len(entities_ORG)

152

In [101]:
len(entities_LOC)

353

Seejärel tuleb tekitada kõigi nimeolemiliikide sagedusloendid:

In [102]:
entities_PER_lower = [name.lower() for name in entities_PER]
entities_ORG_lower = [name.lower() for name in entities_ORG]
entities_LOC_lower = [name.lower() for name in entities_LOC]

In [160]:
frequencies_PER = Counter(entities_PER_lower)
frequencies_ORG = Counter(entities_ORG_lower)
frequencies_LOC = Counter(entities_LOC_lower)

In [161]:
print('PER' + '\t' + 'ORG' + '\t' + 'LOC')
for (per, org, loc) in itertools.zip_longest(frequencies_PER.most_common(), frequencies_ORG.most_common(), frequencies_LOC.most_common()):
    print(per, org, loc)

PER	ORG	LOC
('edgar savisaar', 17) ('keskerakond', 20) ('eesti', 98)
('jüri', 10) ('rein', 8) ('tallinn', 45)
('andrus ansip', 8) ('irl', 7) ('euroopa', 18)
('ansip', 8) ('euroopa liit', 5) ('hiina', 15)
('kaarel', 8) ('reformierakond', 5) ('tartu', 10)
('rein', 7) ('euroopa liidu', 5) ('soome', 8)
('andrus', 7) ('kapo', 4) ('rootsi', 4)
('marek strandberg', 6) ('riigikogu', 4) ('läti', 4)
('allar jõks', 5) ('eesti energia', 3) ('euroopa liit', 4)
('strandberg', 5) ('bns', 3) ('eesti vabariik', 3)
('hans', 4) ('tallinn tehnikaülikool', 3) ('narva', 3)
('sven', 4) ('isamaa', 3) ('eesti riik', 3)
('andres', 4) ('briti parlament', 3) ('loksa', 3)
('toomas hendrik', 4) ('h', 2) ('tallinn linn', 3)
('oviir', 4) ('delfi', 2) ('pariis', 3)
('mihkel oviir', 4) ('tallinn ülikool', 2) ('kreeka', 3)
('ainar ruussaar', 4) ('el', 2) ('pärnu', 3)
('gunnar', 3) ('kreeta', 2) ('helsingi', 2)
('indrek', 3) ('nato', 2) ('keila', 2)
('tiit', 3) ('tartu rein', 2) ('brüssel', 2)
('aivar', 3) ('postimees', 

Loon uued järjendid isiku-, organisatsiooni- ja asukohanimede jaoks:

In [162]:
entities_PER_new = []
entities_ORG_new = []
entities_LOC_new = []

In [163]:
frequencies_PER_copy = deepcopy(frequencies_PER)
frequencies_ORG_copy = deepcopy(frequencies_ORG)
frequencies_LOC_copy = deepcopy(frequencies_LOC)

In [164]:
equal_occurrences = []

In [165]:
#Kas isikunimede hulgas olevad nimed leiduvad ka organisatsiooni- või asukohanimede hulgas:
for name in frequencies_PER_copy.keys():
    if frequencies_ORG_copy[name] > 0:
        if frequencies_ORG_copy[name] > frequencies_PER_copy[name]:
            print(name, 'ORG:', frequencies_ORG_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_PER[name]
        elif frequencies_ORG_copy[name] < frequencies_PER_copy[name]:
            print(name, 'ORG:', frequencies_ORG_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_ORG[name]
        else:
            equal_occurrences.append(name)
                
    if frequencies_LOC_copy[name] > 0:
        if frequencies_LOC_copy[name] > frequencies_PER_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_PER[name]
        elif frequencies_LOC_copy[name] < frequencies_PER_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'PER:', frequencies_PER_copy[name])
            del frequencies_LOC[name]
        else:
            equal_occurrences.append(name)
            
#Kas organisatsiooninimede hulgas leidub nimesid, mis esinevad ka asukohanimede hulgas (isikunimesid pole vaja uuesti läbi käia):
for name in frequencies_ORG_copy.keys():
    if frequencies_LOC_copy[name] > 0:
        if frequencies_LOC_copy[name] > frequencies_ORG_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'ORG:', frequencies_ORG_copy[name])
            del frequencies_ORG[name]
        elif frequencies_LOC_copy[name] < frequencies_ORG_copy[name]:
            print(name, 'LOC:', frequencies_LOC_copy[name], 'ORG:', frequencies_ORG_copy[name])
            del frequencies_LOC[name]
        else:
            equal_occurrences.append(name)
            
#Asukohanimesid pole vaja eraldi läbi käia, seal leiduvad duplikaadid kas isiku- või organisatsiooninimedest on juba tuvastatud.  

rein ORG: 8 PER: 7
euroopa liit LOC: 4 ORG: 5


In [166]:
print('PER' + '\t' + 'ORG' + '\t' + 'LOC')
for (per, org, loc) in itertools.zip_longest(frequencies_PER.most_common(), frequencies_ORG.most_common(), frequencies_LOC.most_common()):
    print(per, org, loc)

PER	ORG	LOC
('edgar savisaar', 17) ('keskerakond', 20) ('eesti', 98)
('jüri', 10) ('rein', 8) ('tallinn', 45)
('andrus ansip', 8) ('irl', 7) ('euroopa', 18)
('ansip', 8) ('euroopa liit', 5) ('hiina', 15)
('kaarel', 8) ('reformierakond', 5) ('tartu', 10)
('andrus', 7) ('euroopa liidu', 5) ('soome', 8)
('marek strandberg', 6) ('kapo', 4) ('rootsi', 4)
('allar jõks', 5) ('riigikogu', 4) ('läti', 4)
('strandberg', 5) ('eesti energia', 3) ('eesti vabariik', 3)
('hans', 4) ('bns', 3) ('narva', 3)
('sven', 4) ('tallinn tehnikaülikool', 3) ('eesti riik', 3)
('andres', 4) ('isamaa', 3) ('loksa', 3)
('toomas hendrik', 4) ('briti parlament', 3) ('tallinn linn', 3)
('oviir', 4) ('h', 2) ('pariis', 3)
('mihkel oviir', 4) ('delfi', 2) ('kreeka', 3)
('ainar ruussaar', 4) ('tallinn ülikool', 2) ('pärnu', 3)
('gunnar', 3) ('el', 2) ('helsingi', 2)
('indrek', 3) ('kreeta', 2) ('keila', 2)
('tiit', 3) ('nato', 2) ('brüssel', 2)
('aivar', 3) ('tartu rein', 2) ('mongoolia', 2)
('timberg', 3) ('postimees', 

In [167]:
print(equal_occurrences)

['teder', 'ekre', 'taska', 'levadia']


In [168]:
for name in frequencies_PER.keys():
    entities_PER_new.append(name)
    
for name in frequencies_ORG.keys():
    entities_ORG_new.append(name)
    
for name in frequencies_LOC.keys():
    entities_LOC_new.append(name)

Järgnevalt on vaja luua andmebaasi uus tabel nimeüksuste hoiustamiseks.

In [179]:
con = sqlite3.connect("media_data.db")
cur = con.cursor()

In [181]:
cur.execute("CREATE TABLE named_entities(ID INTEGER PRIMARY KEY, sentence_ID INTEGER, PER TEXT, ORG TEXT)")

In [182]:
for i in range(len(corpus)):
    for ner in corpus[i].ner:
        lemmatized_ner = [word.lemma[0] for word in ner]
        lemmatized_ner_text = ' '.join(lemmatized_ner).lower()
        if ner.nertag == 'PER' and lemmatized_ner_text in entities_PER_new:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, PER)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag != 'PER' and lemmatized_ner_text in entities_PER_new and lemmatized_ner_text not in equal_occurrences:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, PER)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag == 'ORG' and lemmatized_ner_text in entities_ORG_new:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, ORG)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        elif ner.nertag != 'ORG' and lemmatized_ner_text in entities_ORG_new and lemmatized_ner_text not in equal_occurrences:
            cur.execute("""INSERT INTO named_entities
                                    (sentence_ID, ORG)
                                    VALUES (?, ?);""", (sentence_ids[i], lemmatized_ner_text))
        con.commit()

In [183]:
con.close()

Praeguse seisuga ei püüta parandada vigaseks hinnatud ner-olemeid, vaid luua ainult parandatud kujul nimekiri ner-olemitest